필요한 패키지 설치하기

In [1]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 10.8 MB/s 
     |████████████████████████████████| 59 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 36.5 MB/s 
     |████████████████████████████████| 3.3 MB 28.2 MB/s 
     |████████████████████████████████| 895 kB 19.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [33]:
import torch
import numpy as np 
from tqdm import tqdm # 진행도 표시해주는 함수 

from transformers import AutoModel, AutoTokenizer # 자동으로 Pretrained 모델과 토크나이저를 불러와주는 함수 

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model_name = "klue/roberta-large"

# 모델과 토크나이저를 전역 변수로 선언 
model = AutoModel.from_pretrained(model_name)
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

데이터 준비하기

In [3]:
import pandas as pd
import ast

df = pd.read_csv('dataset.csv')

In [4]:
df

,speaker,saying,categories
0,에머슨,인생은 하나의 실험이다. 실험이 많아질수록 당신은 더 좋은 사람이 된다.,"['인생', '경험']"
1,파스칼,"인생은 우주의 영광이요, 또한 우주의 모욕이다.",['인생']
2,W. NL. 영안,"인생은 반복된 생활이다. 좋은 일을 반복하면 좋은 인생을, 나쁜 일을 반복하면 불행...","['인생', '생활', '반복', '습관']"
3,파스칼,인생의 최고 불행은 인간이면서 인간을 모르는 것이다.,"['인생', '불행', '인간']"
4,헉슬리,인생의 위대한 목표는 지식이 아니라 행동이다.,"['인생', '지식', '행동']"
...,...,...,...
13254,조지 버나드 쇼,황금률은 없다는 것이 황금률이다.,['황금률']
13255,『성경』,"악에게 지지 말고, 선으로 악을 이기십시오.","['악(evil)', '선(good)']"
13256,영국 속담,죽은 후에는 사는 자가 아니면 산 사람이라고는 할 수 없다.,"['죽음', '삶']"
13257,A. 쇼펜하워,여성적인 성격의 기본적인 결함은 정의감이 없다는 것이다.,"['여성', '정의']"


In [10]:
class Data:
    def __init__(self, id_,speaker,saying, tags):
        self.id = id_
        self.speaker = speaker
        self.saying = saying
        self.tags = tags

datalist = []

for item in df.itertuples():
    data = Data(item.Index, item.speaker, item.saying, ast.literal_eval(item.categories))
    datalist.append(data)

In [13]:
print("id:", datalist[0].id)
print("speaker:",datalist[0].speaker)
print("saying:",datalist[0].saying )
print("tags:", datalist[0].tags)

id: 0
speaker: 에머슨
saying: 인생은 하나의 실험이다. 실험이 많아질수록 당신은 더 좋은 사람이 된다.
tags: ['인생', '경험']


In [37]:
saying = "인생은 하나의 실험이다. 실험이 많아질수록 당신은 더 좋은 사람이 된다."
tokenized_input = tokenizer(saying, return_tensors="pt")
print(tokenized_input)
# 모델에 Dict형식으로 인풋이 들어감.
'''
{
  'input_ids': tokenized된 문장의 텐서
  'token_type_ids': 무시해도 됨.
  'attention_mask': 이거도 무시해도 됨.
}
''' 
for key,val in tokenized_input.items():
    tokenized_input[key] = val.to(device) # gpu 있으면 gpu에 올리기. 없으면 그냥 cpu에

{'input_ids': tensor([[    0,  4545,  2073,  3657,  2079,  5270, 28674,    18,  5270,  2052,
          1039,  2227,  2431,  4556,  3994,  2073,   831,  1560,  2073,  3611,
          2052,  3622,    18,     2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [38]:
outputs = model(**tokenized_input)
outputs

BaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                               tensor([[[ 0.0821,  0.0707, -0.3700,  ..., -0.1926, -0.0062,  0.5077],
                                                        [ 0.4245, -0.4755, -0.0418,  ..., -0.5073,  0.4965,  0.3767],
                                                        [-0.3929,  0.4948,  0.5466,  ..., -0.0375, -0.0730,  0.3291],
                                                        ...,
                                                        [-0.0530, -0.1398,  0.0446,  ..., -0.0051, -0.3265,  0.5177],
                                                        [ 0.1109, -0.0977, -0.3218,  ..., -0.1198,  0.0012,  0.6569],
                                                        [ 0.1008,  0.0752, -0.3701,  ..., -0.1857, -0.0015,  0.5165]]],
                                                      device='cuda:0', grad_fn=<NativeLayerNormBackward>)),
                                              ('poole

In [39]:
#우리가 필요한 건 outputs의 'pooler_output' <- 이게 입력으로 넣은 문장을 대표하는 벡터임. 
print(outputs.pooler_output)
print(outputs.pooler_output.size())

tensor([[ 0.3049,  0.0287,  0.0650,  ...,  0.0082,  0.1230, -0.2727]],
       device='cuda:0', grad_fn=<TanhBackward>)
torch.Size([1, 1024])


In [40]:
# 텐서를 넘파이 벡터로 바꾸기. 만약 node js에서 벡터 연산을 하려면 관련 라이브러리를 찾아봐야 할듯. 
vector = outputs.pooler_output.detach().squeeze().cpu().numpy()
print(type(vector))
print(vector)
print(vector.shape)

<class 'numpy.ndarray'>
[ 0.304885    0.02867789  0.06504972 ...  0.00815088  0.12302867
 -0.27273422]
(1024,)


위의 과정을 하나의 함수로 요약

In [41]:
def generate_vector(saying: str) -> np.ndarray:
    tokenized_input = tokenizer(saying, return_tensors="pt")
    for key,val in tokenized_input.items(): # gpu 있으면 gpu에 올리기
        tokenized_input[key] = val.to(device)
    outputs = model(**tokenized_input)
    vector = outputs.pooler_output.detach().squeeze().cpu().numpy()
    return vector

In [42]:
datalist[1000].saying

'모든 거짓 중에서 으뜸가는 가장 나쁜 것은 자기자신을 속이는 일이다.'

In [43]:
vector = generate_vector("모든 거짓 중에서 으뜸가는 가장 나쁜 것은 자기자신을 속이는 일이다.")
print(vector)

[ 0.27819386  0.04261396 -0.00635708 ... -0.14844678  0.07506038
 -0.40162823]


모든 명언에 대해 벡터 생성하기

In [44]:
vectors = []

for data in tqdm(datalist):
    saying = data.saying
    vector = generate_vector(saying)
    vectors.append(vector)

100%|██████████| 13259/13259 [09:47<00:00, 22.56it/s]


In [45]:
len(vectors)

13259

In [46]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(a,b): # 코사인 유사도 계싼 함수
    return dot(a, b)/(norm(a)*norm(b)) 

In [47]:
cos_sim(vectors[0],vectors[1])

0.9727473

In [51]:
def top5(target_id):
    target_vector = vectors[target_id]
    cos_sims = []
    for i,vector in enumerate(vectors):
        cos_sims.append((i,cos_sim(target_vector,vector)))
    cos_sims.sort(key=lambda x:x[1], reverse=True)
    
    res = []
    for item in cos_sims:
        if item[0]!=target_id: res.append(item)
        if len(res)==5: break
        
    print("target:",datalist[target_id].saying)
    
    print("candidates:")
    for i, sim in res:
        print(i,datalist[i].saying,sim)
    return res

In [52]:
target_id = 1000
datalist[target_id].saying

'모든 거짓 중에서 으뜸가는 가장 나쁜 것은 자기자신을 속이는 일이다.'

In [53]:
res = top5(target_id)

target: 모든 거짓 중에서 으뜸가는 가장 나쁜 것은 자기자신을 속이는 일이다.
candidates:
9924 결점 중에서 가장 큰 결점은 그것을 깨닫지 못하는 것이다. 0.99804085
3831 학문의 최대의 적은 자기 마음속에 있는 유혹이다. 0.99797887
8599 얌전한 체하는 것은 허욕의 일종이요, 모든 허욕 중에 가장 나쁜 것이다. 0.99793625
10555 어리석은 짓을 삼가는 것이 지혜의 입문이다. 0.9977756
2583 속이는 말로 재물을 모으는 것은 죽음을 구하는 것이다. 0.99768245
